In [262]:
import pandas as pd
from sklearn import svm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

ohe = OneHotEncoder(sparse=False)
# Crie um objeto LabelEncoder
le = LabelEncoder()
# carregar o conjunto de treino
train_data = pd.read_csv('healthcare-dataset-stroke-train.csv')

# carregar o conjunto de teste
test_data = pd.read_csv('healthcare-dataset-stroke-test.csv')
#Aí e para carregar os ficheiros

train_data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,68059,Male,35.0,0,0,Yes,Govt_job,Rural,103.08,41.5,smokes,0
1,18437,Male,26.0,0,0,No,Private,Urban,85.92,35.6,smokes,0
2,23427,Female,81.0,0,0,Yes,Private,Rural,91.82,36.9,Unknown,0
3,59309,Male,18.0,0,0,No,Self-employed,Urban,74.00,23.7,Unknown,0
4,72701,Male,2.0,0,0,No,children,Rural,112.66,14.2,Unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,61178,Male,39.0,0,0,Yes,Private,Urban,164.67,33.8,Unknown,0
4496,56575,Female,51.0,1,0,Yes,Govt_job,Urban,69.94,33.3,smokes,0
4497,25636,Male,40.0,0,0,Yes,Private,Rural,201.96,30.1,Unknown,0
4498,41827,Male,58.0,0,0,Yes,Private,Rural,135.89,23.1,formerly smoked,0


In [263]:
train_data.info

<bound method DataFrame.info of          id  gender   age  hypertension  heart_disease ever_married   
0     68059    Male  35.0             0              0          Yes  \
1     18437    Male  26.0             0              0           No   
2     23427  Female  81.0             0              0          Yes   
3     59309    Male  18.0             0              0           No   
4     72701    Male   2.0             0              0           No   
...     ...     ...   ...           ...            ...          ...   
4495  61178    Male  39.0             0              0          Yes   
4496  56575  Female  51.0             1              0          Yes   
4497  25636    Male  40.0             0              0          Yes   
4498  41827    Male  58.0             0              0          Yes   
4499  53266  Female  33.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi   smoking_status   
0          Govt_job          Rural 

In [264]:
train_data = train_data.loc[train_data['smoking_status'] != 'Unknown']
train_data = train_data.loc[train_data['bmi'] != 'NaN']
train_data = train_data.loc[train_data['hypertension'] != 'Unknown']
train_data = train_data.loc[train_data['avg_glucose_level'] != 'NaN']
train_data = train_data.loc[train_data['work_type'] != 'Unknown']
train_data = train_data.loc[train_data['Residence_type'] != 'Unknown']
train_data = train_data.loc[train_data['avg_glucose_level'] != 'NaN']
train_data = train_data.loc[train_data['ever_married'] != 'Unknown']
train_data = train_data.loc[train_data['gender'] != 'Unknown']
train_data.dropna(inplace=True)

In [265]:
train_data.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [266]:
train_data.gender = train_data.gender.astype('category')
train_data.ever_married = train_data.ever_married.astype('category')
train_data.work_type = train_data.work_type.astype('category')
train_data.Residence_type = train_data.Residence_type.astype('category')
train_data.smoking_status = train_data.smoking_status.astype('category')

cat_columns = train_data.select_dtypes(['category']).columns
train_data[cat_columns] = train_data[cat_columns].apply(lambda x: pd.factorize(x)[0])

In [267]:
train_data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,68059,0,35.0,0,0,0,0,0,103.08,41.5,0,0
1,18437,0,26.0,0,0,1,1,1,85.92,35.6,0,0
5,4083,1,30.0,0,0,1,1,0,73.69,17.3,1,0
6,34188,1,47.0,0,0,0,0,1,95.07,38.8,2,0
7,67112,1,56.0,0,0,0,1,0,77.66,40.8,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4493,5068,1,28.0,0,0,1,1,1,76.81,28.3,0,0
4494,56594,1,70.0,0,0,0,1,1,77.77,33.8,0,0
4496,56575,1,51.0,1,0,0,0,1,69.94,33.3,0,0
4498,41827,0,58.0,0,0,0,1,0,135.89,23.1,2,0


In [268]:
X = train_data.drop(columns=["stroke"])
y = train_data['stroke']

In [269]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [270]:
svm_clf = svm.SVC()
scores = cross_val_score(svm_clf, X_train, y_train, cv=5)
print(scores)

[0.94911504 0.94911504 0.95121951 0.95121951 0.94900222]


In [271]:
lr_clf = LogisticRegression(random_state=0, max_iter=5000)
scores = cross_val_score(lr_clf, X_train, y_train, cv=5)
print(scores)

[0.94469027 0.94911504 0.95121951 0.95121951 0.94900222]


In [272]:
tree_clf = tree.DecisionTreeClassifier()
scores = cross_val_score(tree_clf, X_train, y_train, cv=5)
print(scores)

[0.9159292  0.8960177  0.89356984 0.90909091 0.89135255]


In [273]:
gnb_clf = GaussianNB()
scores = cross_val_score(gnb_clf, X_train, y_train, cv=5)
print(scores)

[0.89823009 0.94469027 0.94456763 0.92682927 0.92682927]


In [274]:
gnb_clf.fit(X_train, y_train)
y_pred = gnb_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Saudável', 'Não Saudável']))

              precision    recall  f1-score   support

    Saudável       0.95      0.97      0.96       707
Não Saudável       0.24      0.15      0.19        46

    accuracy                           0.92       753
   macro avg       0.59      0.56      0.57       753
weighted avg       0.90      0.92      0.91       753



In [275]:
tree_clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
y_pred = tree_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Saudável', 'Não Saudável']))

              precision    recall  f1-score   support

    Saudável       0.94      0.94      0.94       707
Não Saudável       0.13      0.13      0.13        46

    accuracy                           0.90       753
   macro avg       0.54      0.54      0.54       753
weighted avg       0.89      0.90      0.89       753



In [276]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Saudável', 'Não Saudável']))

              precision    recall  f1-score   support

    Saudável       0.94      1.00      0.97       707
Não Saudável       0.67      0.04      0.08        46

    accuracy                           0.94       753
   macro avg       0.80      0.52      0.53       753
weighted avg       0.92      0.94      0.91       753

